In [1]:
from pymongo import MongoClient
from src.get_from_mongo import get_data
import numpy as np
import pickle
import tensorflow as tf

In [2]:
id_from_char = {
    'CAPTAIN_FALCON' : 1 ,
    'DONKEY_KONG'    : 2 ,
    'FOX'            : 3 ,
    'GAME_AND_WATCH' : 4 ,
    'KIRBY'          : 5 ,
    'BOWSER'         : 6 ,
    'LINK'           : 7 ,
    'LUIGI'          : 8 ,
    'MARIO'          : 9 ,
    'MARTH'          : 10 ,
    'MEWTWO'         : 11 ,
    'NESS'           : 12 ,
    'PEACH'          : 13 ,
    'PIKACHU'        : 14 ,
    'ICE_CLIMBERS'   : 15 ,
    'JIGGLYPUFF'     : 16 ,
    'SAMUS'          : 17 ,
    'YOSHI'          : 18 ,
    'ZELDA'          : 19 ,
    'SHEIK'          : 20 ,
    'FALCO'          : 21 ,
    'YOUNG_LINK'     : 22 ,
    'DR_MARIO'       : 23 ,
    'ROY'            : 24 ,
    'PICHU'          : 25 ,
    'GANONDORF'      : 26 ,
}

char_from_id = {v:k for k, v in id_from_char.items()}

In [3]:
database_name = 'slippi'
collection_name = 'Blynde'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [4]:
query = {}
bytestreams, characters = get_data(database_name, collection_name)
print(f'Documents retrieved: {len(characters)}')

Documents retrieved: 1816


In [5]:
x = []
y = []
for bytestream, character in zip(bytestreams, characters):
    istream = pickle.loads(bytestream).toarray()
    T = 30     # clip length in seconds
    F = T * 60 # clip length in frames
    f = 0      # clip starting frame
    while f+F < istream.shape[0]:
        y.append(id_from_char[character])
        x.append(istream[f:f+F])
        f += F

X = np.stack(x, axis=0)
Y = tf.one_hot(y, 26)

In [6]:
X_train, Y_train = X[:-500], Y[:-500]
X_test, Y_test = X[-500:], Y[-500:]

In [7]:
X.shape

(9798, 1800, 13)

In [8]:
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, InputLayer
from tensorflow.keras.layers import Conv1D, MaxPooling1D

In [9]:
model = Sequential()

model.add(InputLayer(input_shape=X.shape[1:]))

model.add(Dropout(.2))

# first conv layer
model.add(Conv1D(100, #num of features extracted from istream
                 60, #number of frames filter can see at once
                 activation='relu'))

model.add(Dropout(.2))

model.add(MaxPooling1D(pool_size=5))

model.add(Conv1D(60,
                 30,
                 activation='relu'))

model.add(MaxPooling1D(pool_size=5))

model.add(Conv1D(60,
                 30,
                 activation='relu'))

model.add(MaxPooling1D(pool_size=5))

model.add(Flatten())

model.add(Dense(40, activation='relu'))

model.add(Dropout(.2))

# final output layer
model.add(Dense(26, activation='softmax'))
                
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 1800, 13)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1741, 100)         78100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1741, 100)         0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 348, 100)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 319, 60)           180060    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 63, 60)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 34, 60)            1

In [39]:
# during fit process watch train and test error simultaneously
model.fit(X_train, Y_train, batch_size=100, epochs=10,
          verbose=1, validation_data=(X_test, Y_test))

score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', round(score[0], 3))
print(f'Test accuracy: {round(score[1]*100)}%')  # this is the one we care about

Test score: 0.675
Test accuracy: 82%


In [12]:
# bytestreams, characters = get_data('slippi', 'gh0st')
# print(f'Documents retrieved: {len(characters)}')

In [13]:
# from tensorflow.keras.utils import to_categorical
# x = []
# y = []
# for bytestream, character in zip(bytestreams, characters):
#     istream = pickle.loads(bytestream).toarray()
#     T = 15     # clip length in seconds
#     F = T * 60 # clip length in frames
#     f = 0      # clip starting frame
#     while f+F < istream.shape[0]:
#         y.append(get_char_id[character])
#         x.append(istream[f:f+F])
#         f += F
        

# X_ramon = np.stack(x, axis=0)
# Y_ramon = tf.one_hot(y, 26)

In [406]:
def test(i, ii):
    character = characters[i]
    istream = pickle.loads(bytestreams[i])

    y_test = []
    x_test = []
    f=0

    while f+F < istream.shape[0]:
        y_test.append(id_from_char[character])
        x_test.append(istream[f:f+F])
        f += F
        
    try:
        d = x_test[ii].toarray()
    except IndexError:
        test(i, ii-1)
        return
    
    pred = char_from_id[np.argmax(model.predict(d.reshape(1,d.shape[0],d.shape[1])))]


    print(f'''
    Actual Character:
    ---------------
    {char_from_id[y_test[ii]]}
    ''')

    print(f'''
    Detected Character:
    ---------------
    {pred}
    ''')

In [409]:
# i = 602 # use this to check different entries
i = 630 # use this to check different entries
ii = 3 # use this to look at different clips from that entry

In [745]:
print(characters[i], '\n', f'i={i}')

PEACH 
 i=797


In [746]:
test(i, ii)
i+=1


    Actual Character:
    ---------------
    PEACH
    

    Detected Character:
    ---------------
    PEACH
    


In [405]:
i-=1